<a href="https://colab.research.google.com/github/lunaczp/alpaca-lora/blob/main/train_in_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed May 29 23:51:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              48W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
! export WANDB_PROJECT=lora-light

In [4]:
!git clone https://github.com/lunaczp/alpaca-lora.git
%cd alpaca-lora

Cloning into 'alpaca-lora'...
remote: Enumerating objects: 552, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 552 (delta 9), reused 9 (delta 3), pack-reused 533
Receiving objects: 100% (552/552), 27.83 MiB | 13.73 MiB/s, done.
Resolving deltas: 100% (323/323), done.
/content/alpaca-lora


In [ ]:
!pip install -r requirements.txt

In [ ]:
!python finetune.py \
    --base_model 'NousResearch/Llama-2-7b-hf' \
    --data_path 'alpaca_data_gpt4.json' \
    --output_dir './lora-alpaca' \
    --batch_size 128 \
    --micro_batch_size 4 \
    --num_epochs 1 \
    --learning_rate 1e-4 \
    --cutoff_len 512 \
    --val_set_size 2000 \
    --lora_r 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --lora_target_modules '[q_proj,v_proj]' \
    --train_on_inputs \
    --group_by_length

In [ ]:
! pip install -U "huggingface_hub[cli]"

In [ ]:
! huggingface-cli login
! cd ./lora-alpaca && huggingface-cli upload LightXXXXX/llama-2-7b-hf-lora-alpaca-light .

In [ ]:
#test
! python generate.py \
    --load_8bit \
    --base_model 'NousResearch/Llama-2-7b-hf' \
    --lora_weights 'LightXXXXX/llama-2-7b-hf-lora-alpaca-light'


In [ ]:
# merge
! export BASE_MODEL='NousResearch/Llama-2-7b-hf'
! export LORA_MODEL='LightXXXXX/llama-2-7b-hf-lora-alpaca-light'
! python export_hf_checkpoint.py

In [ ]:
#upload
! cd ./hf_ckpt && huggingface-cli upload LightXXXXX/llama-2-7b-hf-alpaca-light . .

In [ ]:
# convert to gguf
%cd /content/
! git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp
! pip install -r requirements.txt
%cd ../


In [ ]:
#build f16
! mkdir models
! python3 llama.cpp/convert.py --outtype f16 --outfile models/llama-2-7b-alpaca-light-f16.gguf hf_ckpt/

In [ ]:
# quantize
%cd llama.cpp
! make
! ./quantize ../models/llama-2-7b-alpaca-light-f16.gguf ../models/llama-2-7b-alpaca-light-Q4_K_M.gguf Q4_K_M
%cd ../

In [ ]:
# upload to huggingface
%cd models
! huggingface-cli upload LightXXXXX/llama-2-7b-alpaca-light-gguf . .